In [16]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from time import sleep 

In [9]:
# Función para obtener detalles generales del juego
def obtener_detalles_juego(id_juego):
    url = f'https://www.boardgamegeek.com/xmlapi2/thing?id={id_juego}&stats=1'
    respuesta = requests.get(url)
    
    if respuesta.status_code == 200:
        xml_root = ET.fromstring(respuesta.content)
        
        # Información general del juego
        name = xml_root.find('item/name').attrib['value']
        year_published = xml_root.find('item/yearpublished').attrib['value']
        description = xml_root.find('item/description').text
        
        # Limpiar la descripción
        description = description.replace('\n', ' ').replace('&quot;', '"').replace('&amp;', '&')
        
        min_players = xml_root.find('item/minplayers').attrib['value']
        max_players = xml_root.find('item/maxplayers').attrib['value']
        min_playtime = xml_root.find('item/minplaytime').attrib['value']
        max_playtime = xml_root.find('item/maxplaytime').attrib['value']
        
        # Valoraciones
        stats = xml_root.find('item/statistics/ratings')
        avg_rating = stats.find('average').attrib['value']
        bayes_avg_rating = stats.find('bayesaverage').attrib['value']
        num_ratings = stats.find('usersrated').attrib['value']
        
        # Guardamos los datos en un diccionario con nombres de columnas en inglés
        game = {
            'BGGId': id_juego,
            'Name': name,
            'Year_Published': year_published,
            'Description': description,
            'Min_Players': min_players,
            'Max_Players': max_players,
            'Min_Playtime': min_playtime,
            'Max_Playtime': max_playtime,
            'Average_Rating': avg_rating,
            'Bayesian_Average_Rating': bayes_avg_rating,
            'Number_of_Ratings': num_ratings
        }
        
        return game
    else:
        print(f"Error al obtener detalles del juego con ID {id_juego}")
        return None



# Función para obtener mecánicas del juego
def obtener_mecanicas(id_juego):
    url = f'https://www.boardgamegeek.com/xmlapi2/thing?id={id_juego}'
    respuesta = requests.get(url)
    
    if respuesta.status_code == 200:
        xml_root = ET.fromstring(respuesta.content)
        
        # Extraer mecánicas del juego
        mecanicas = []
        
        for link in xml_root.findall('item/link'):
            link_type = link.attrib.get('type')
            link_value = link.attrib.get('value')
            
            if link_type == 'boardgamemechanic':
                mecanicas.append(link_value)
        
        return {'BGGId': id_juego, 'Mecánicas': mecanicas}
    
    else:
        print(f"Error al obtener mecánicas del juego con ID {id_juego}")
        return None
    



    # Función para obtener categorías del juego
def obtener_categorias(id_juego):
    url = f'https://www.boardgamegeek.com/xmlapi2/thing?id={id_juego}'
    respuesta = requests.get(url)
    
    if respuesta.status_code == 200:
        xml_root = ET.fromstring(respuesta.content)
        
        # Extraer categorías del juego
        categorias = []
        
        for link in xml_root.findall('item/link'):
            link_type = link.attrib.get('type')
            link_value = link.attrib.get('value')
            
            if link_type == 'boardgamecategory':
                categorias.append(link_value)
        
        return {'BGGId': id_juego, 'Categorías': categorias}
    
    else:
        print(f"Error al obtener categorías del juego con ID {id_juego}")
        return None

In [17]:
# Recopilación de juegos masivos
ids_juegos = range(1, 10000)  # Ejemplo: IDs de 1 a 9999 (ajusta el rango según lo que necesites)

datos_generales = []
for id_juego in ids_juegos:
    detalles_juego = obtener_detalles_juego(id_juego)
    if detalles_juego:
        datos_generales.append(detalles_juego)
    
    sleep(1)  # Pausar 1 segundo entre solicitudes para evitar ser bloqueado

# Crear un DataFrame con los datos generales
df_generales = pd.DataFrame(datos_generales)

AttributeError: 'NoneType' object has no attribute 'attrib'

In [10]:
# Lista para almacenar los datos generales de los juegos
datos_generales = []

# IDs de los juegos que queremos obtener
ids_juegos = [13, 174430, 68448, 12333]  # Ejemplos de IDs

# Iterar sobre los IDs y obtener los datos generales
for id_juego in ids_juegos:
    detalles_juego = obtener_detalles_juego(id_juego)
    if detalles_juego:
        datos_generales.append(detalles_juego)

# Crear un DataFrame de los datos generales
df_generales = pd.DataFrame(datos_generales)

# Mostrar el DataFrame resultante
df_generales

,BGGId,Name,Year_Published,Description,Min_Players,Max_Players,Min_Playtime,Max_Playtime,Average_Rating,Bayesian_Average_Rating,Number_of_Ratings
0,13,CATAN,1995,"In CATAN (formerly The Settlers of Catan), pla...",3,4,60,120,7.09697,6.9188,129877
1,174430,Gloomhaven,2017,Gloomhaven is a game of Euro-inspired tactica...,1,4,60,120,8.58029,8.34433,63087
2,68448,7 Wonders,2010,You are the leader of one of the 7 great citie...,2,7,30,30,7.6797,7.56953,105916
3,12333,Twilight Struggle,2005,"""Now the trumpet summons us again, not as a ca...",2,2,120,180,8.24349,8.06449,49329


In [11]:
# Inicializar el DataFrame de mecánicas con la columna BGGId
df_mecanicas = pd.DataFrame({'BGGId': ids_juegos})

# Iterar sobre los IDs y obtener las mecánicas
for id_juego in ids_juegos:
    mecanicas_juego = obtener_mecanicas(id_juego)
    
    if mecanicas_juego:
        # Para cada mecánica encontrada, crear una columna si no existe
        for mecanica in mecanicas_juego['Mecánicas']:
            # Limpiar el nombre de la mecánica (remover espacios o caracteres especiales)
            col_name = mecanica.strip().replace(' ', '_').replace('-', '_')
            if col_name not in df_mecanicas.columns:
                df_mecanicas[col_name] = 0  # Inicializar con 0 en todos los juegos existentes
            df_mecanicas.loc[df_mecanicas['BGGId'] == id_juego, col_name] = 1  # Marcar la mecánica con un 1

# Llenar los valores faltantes con 0
df_mecanicas.fillna(0, inplace=True)

# Convertir las columnas binarias a int
df_mecanicas = df_mecanicas.astype(int)

# Mostrar el DataFrame de mecánicas
df_mecanicas


,BGGId,Chaining,Dice_Rolling,Hexagon_Grid,Income,Modular_Board,Negotiation,Network_and_Route_Building,Race,Random_Production,...,Closed_Drafting,Neighbor_Scope,Set_Collection,Action/Event,Advantage_Token,Area_Majority_/_Influence,Events,Simulation,Sudden_Death_Ending,Tug_of_War
0,13,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,174430,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,68448,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
3,12333,0,1,0,0,0,0,0,0,0,...,0,0,0,1,1,1,1,1,1,1


In [12]:
df_mecanicas.columns

Index(['BGGId', 'Chaining', 'Dice_Rolling', 'Hexagon_Grid', 'Income',
       'Modular_Board', 'Negotiation', 'Network_and_Route_Building', 'Race',
       'Random_Production', 'Trading', 'Variable_Set_up', 'Action_Queue',
       'Action_Retrieval', 'Campaign_/_Battle_Card_Driven',
       'Card_Play_Conflict_Resolution', 'Communication_Limits',
       'Cooperative_Game', 'Critical_Hits_and_Failures', 'Deck_Construction',
       'Grid_Movement', 'Hand_Management', 'Legacy_Game', 'Line_of_Sight',
       'Multi_Use_Cards', 'Narrative_Choice_/_Paragraph',
       'Once_Per_Game_Abilities', 'Role_Playing',
       'Scenario_/_Mission_/_Campaign_Game', 'Simultaneous_Action_Selection',
       'Solo_/_Solitaire_Game', 'Storytelling', 'Tags',
       'Variable_Player_Powers', 'Closed_Drafting', 'Neighbor_Scope',
       'Set_Collection', 'Action/Event', 'Advantage_Token',
       'Area_Majority_/_Influence', 'Events', 'Simulation',
       'Sudden_Death_Ending', 'Tug_of_War'],
      dtype='object')

In [13]:
# Inicializar el DataFrame de categorías con la columna BGGId
df_categorias = pd.DataFrame({'BGGId': ids_juegos})

# Iterar sobre los IDs y obtener las categorías
for id_juego in ids_juegos:
    categorias_juego = obtener_categorias(id_juego)
    
    if categorias_juego:
        # Para cada categoría encontrada, crear una columna si no existe
        for categoria in categorias_juego['Categorías']:
            # Limpiar el nombre de la categoría (remover espacios o caracteres especiales)
            col_name = categoria.strip().replace(' ', '_').replace('-', '_')
            if col_name not in df_categorias.columns:
                df_categorias[col_name] = 0  # Inicializar con 0 en todos los juegos existentes
            df_categorias.loc[df_categorias['BGGId'] == id_juego, col_name] = 1  # Marcar la categoría con un 1

# Llenar los valores faltantes con 0
df_categorias.fillna(0, inplace=True)

# Convertir las columnas binarias a int
df_categorias = df_categorias.astype(int)

# Mostrar el DataFrame de categorías
df_categorias


,BGGId,Economic,Negotiation,Adventure,Exploration,Fantasy,Fighting,Miniatures,Ancient,Card_Game,City_Building,Civilization,Modern_Warfare,Political,Wargame
0,13,1,1,0,0,0,0,0,0,0,0,0,0,0,0
1,174430,0,0,1,1,1,1,1,0,0,0,0,0,0,0
2,68448,1,0,0,0,0,0,0,1,1,1,1,0,0,0
3,12333,0,0,0,0,0,0,0,0,0,0,0,1,1,1


In [14]:
df_categorias.columns

Index(['BGGId', 'Economic', 'Negotiation', 'Adventure', 'Exploration',
       'Fantasy', 'Fighting', 'Miniatures', 'Ancient', 'Card_Game',
       'City_Building', 'Civilization', 'Modern_Warfare', 'Political',
       'Wargame'],
      dtype='object')